**Пропуски и дубликаты**

In [182]:
import pandas as pd

In [183]:
data=pd.read_csv('tripadvisor_yelp_opentable_data.csv')

В некоторых столбцах -1 вместо None. Исправим для удобных преобразований с дубликатами (потом вернем)

In [184]:
data['reviews_rating_1'] = data['reviews_rating_1'].where(data['reviews_rating_1'] >= 0, None)
data['reviews_rating_2'] = data['reviews_rating_2'].where(data['reviews_rating_2'] >= 0, None)
data['reviews_rating_3'] = data['reviews_rating_3'].where(data['reviews_rating_3'] >= 0, None)
data['reviews_rating_4'] = data['reviews_rating_4'].where(data['reviews_rating_4'] >= 0, None)
data['reviews_rating_5'] = data['reviews_rating_5'].where(data['reviews_rating_5'] >= 0, None)

Найдем дубликаты

In [185]:
data=data.drop_duplicates()

In [186]:
data_duplicated = data['name'].duplicated()
duplicates_name_indexes=list(data_duplicated[data_duplicated==True].reset_index()['index'])

In [187]:
duplicates_name = list(data.loc[data.index.isin(duplicates_name_indexes)==True]['name']) #лист с названиями ресторанов у которых есть дубликаты

In [188]:
data_duplicates=data[data['name'].isin(duplicates_name)==True].sort_values(by='name').reset_index()
data_duplicates.head()

,index,id,name,reviews,rating,price_range,is_sponsored,latitude,longitude,has_delivery,is_premium,cuisines,diets,dining_options,reviews_rating_1,reviews_rating_2,reviews_rating_3,reviews_rating_4,reviews_rating_5
0,8161,24050169,6 Hamilton Place,6,4.0,$$$$,False,51.504646,-0.150433,True,False,"Chinese, Lebanese, Seafood, Middle Eastern, Pe...",NaN,"Reservations, Private Dining, Seating, Serves ...",0.0,2.0,0.0,0.0,4.0
1,10695,NaN,6 Hamilton Place,270,4.7,££££,NaN,NaN,NaN,False,NaN,"Chinese, Middle Eastern, Lebanese","Vegan, Gluten-free Options","Beer, Cocktails, Corkage Fee, Full Bar, Gender...",NaN,NaN,NaN,NaN,NaN
2,2707,796277,A Toca Restaurant,368,4.0,$$ - $$$,False,51.475372,-0.131148,True,False,"Mediterranean, European, Portuguese","Vegetarian friendly, Gluten free options","Takeout, Reservations, Seating, Television, Hi...",22.0,14.0,30.0,105.0,197.0
3,10383,53aHpjWCGF6znM1P3gVKcQ,A Toca Restaurant,20,4.2,££,NaN,51.475368,-0.131174,NaN,NaN,Portuguese,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3869,911355,ASK Italian,1158,4.0,$$ - $$$,False,51.494873,-0.183515,True,False,"Italian, Pizza, Mediterranean, European","Vegetarian friendly, Vegan options, Gluten fre...","Delivery, Takeout, Reservations, Seating, High...",68.0,77.0,197.0,442.0,374.0


Заполним в исходной таблице строки с недостающими значениями с помощью добавления информации из дубликатов по name.

In [189]:
# Группировка индексов по name
for name, group in data_duplicates.groupby('name'):
    indexes = group['index'].tolist()

    # Извлекаем строки-дубликаты
    duplicates = data.loc[indexes]

    # Фильтруем строки с непустым latitude
    non_empty_latitude = duplicates[duplicates['latitude'].notnull()]

    # Проверяем, есть ли такие строки
    if not non_empty_latitude.empty:
        # Выбираем первую строку с непустым latitude
        filled_row = non_empty_latitude.iloc[0]

        # Заполняем пустые значения в каждой строке-дубликате
        for idx in indexes:
            data.loc[idx] = data.loc[idx].combine_first(filled_row)

data.sort_values(by='name')

,id,name,reviews,rating,price_range,is_sponsored,latitude,longitude,has_delivery,is_premium,cuisines,diets,dining_options,reviews_rating_1,reviews_rating_2,reviews_rating_3,reviews_rating_4,reviews_rating_5
1132,11689384,'O ver,515,4.5,$$ - $$$,False,51.504765,-0.093871,True,False,"Italian, Neapolitan","Vegetarian friendly, Vegan options, Gluten fre...","Delivery, Takeout, Reservations, Seating, Whee...",19.0,17.0,49.0,108.0,322.0
7403,11880271,01 Adana Restaurant,32,3.5,$$ - $$$,False,51.552500,-0.086858,True,False,"Mediterranean, European, Turkish","Vegetarian friendly, Halal","Delivery, Takeout, Reservations, Seating, Tabl...",10.0,0.0,1.0,3.0,18.0
6659,23379654,081 Pizzeria London,25,4.5,NaN,False,51.470905,-0.069088,False,False,"Italian, Pizza, Neapolitan, Campania, Southern...",NaN,"Delivery, Takeout, Reservations, Outdoor Seati...",2.0,1.0,0.0,0.0,22.0
891,697997,1 Lombard Street,616,4.0,$$$$,False,51.512780,-0.088908,False,False,French,"Vegetarian friendly, Vegan options, Gluten fre...","Reservations, Private Dining, Seating, Serves ...",27.0,56.0,109.0,221.0,204.0
1245,2587004,10 Greek Street,594,4.5,$$ - $$$,False,51.514473,-0.131098,False,False,European,"Vegetarian friendly, Vegan options, Gluten fre...","Seating, Serves Alcohol, Full Bar, Accepts Cre...",11.0,11.0,39.0,185.0,348.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4238,2550648,wagamama north greenwich o2,706,4.0,$$ - $$$,False,51.500750,0.005635,True,False,"Japanese, Asian, Soups","Vegetarian friendly, Vegan options, Gluten fre...","Takeout, Reservations, Seating, Highchairs Ava...",42.0,49.0,97.0,285.0,233.0
7619,720906,wagamama wigmore street,379,3.5,$$ - $$$,False,51.515620,-0.152170,True,False,"Japanese, Asian","Vegetarian friendly, Vegan options, Gluten fre...","Delivery, Takeout, Seating, Highchairs Availab...",48.0,30.0,72.0,118.0,111.0
4825,13962474,yamas Greek Cuisine,93,4.5,$$ - $$$,False,51.535713,-0.139406,False,False,"Mediterranean, Greek","Vegetarian friendly, Vegan options, Gluten fre...","Takeout, Reservations, Seating, Serves Alcohol...",0.0,4.0,12.0,12.0,65.0
2158,12922934,Ô Gourmet Libanais,151,5.0,$$ - $$$,False,51.463364,-0.184632,True,False,"Lebanese, Mediterranean, Middle Eastern","Vegetarian friendly, Vegan options, Halal, Glu...","Delivery, Takeout, Reservations, Outdoor Seati...",1.0,0.0,2.0,8.0,140.0


Посмотрим что поменялось для таблицы дубликатов

In [190]:
data_duplicates=data[data['name'].isin(duplicates_name)==True].sort_values(by='name').reset_index()
data_duplicates.head(5)

,index,id,name,reviews,rating,price_range,is_sponsored,latitude,longitude,has_delivery,is_premium,cuisines,diets,dining_options,reviews_rating_1,reviews_rating_2,reviews_rating_3,reviews_rating_4,reviews_rating_5
0,8161,24050169,6 Hamilton Place,6,4.0,$$$$,False,51.504646,-0.150433,True,False,"Chinese, Lebanese, Seafood, Middle Eastern, Pe...",NaN,"Reservations, Private Dining, Seating, Serves ...",0.0,2.0,0.0,0.0,4.0
1,10695,24050169,6 Hamilton Place,270,4.7,££££,False,51.504646,-0.150433,False,False,"Chinese, Middle Eastern, Lebanese","Vegan, Gluten-free Options","Beer, Cocktails, Corkage Fee, Full Bar, Gender...",0.0,2.0,0.0,0.0,4.0
2,2707,796277,A Toca Restaurant,368,4.0,$$ - $$$,False,51.475372,-0.131148,True,False,"Mediterranean, European, Portuguese","Vegetarian friendly, Gluten free options","Takeout, Reservations, Seating, Television, Hi...",22.0,14.0,30.0,105.0,197.0
3,10383,53aHpjWCGF6znM1P3gVKcQ,A Toca Restaurant,20,4.2,££,False,51.475368,-0.131174,True,False,Portuguese,"Vegetarian friendly, Gluten free options","Takeout, Reservations, Seating, Television, Hi...",22.0,14.0,30.0,105.0,197.0
4,3869,911355,ASK Italian,1158,4.0,$$ - $$$,False,51.494873,-0.183515,True,False,"Italian, Pizza, Mediterranean, European","Vegetarian friendly, Vegan options, Gluten fre...","Delivery, Takeout, Reservations, Seating, High...",68.0,77.0,197.0,442.0,374.0


Теперь строки заполнены по максимуму. но от дубликатов по прежнему не избавились

In [191]:
data.head()

,id,name,reviews,rating,price_range,is_sponsored,latitude,longitude,has_delivery,is_premium,cuisines,diets,dining_options,reviews_rating_1,reviews_rating_2,reviews_rating_3,reviews_rating_4,reviews_rating_5
0,2418348,Leicester Arms,602,4.5,$$ - $$$,True,51.510597,-0.137677,False,False,"British, Pub","Vegetarian friendly, Gluten free options","Reservations, Seating, Street Parking, Televis...",30.0,23.0,51.0,156.0,342.0
1,26452946,Swiss Butter,182,4.0,$$ - $$$,True,51.520620,-0.122808,False,True,"French, Steakhouse",NaN,"Seating, Wheelchair Accessible, Serves Alcohol...",21.0,15.0,14.0,22.0,110.0
2,25151198,Florattica Rooftop Bar London,316,4.5,$$ - $$$,False,51.512955,-0.075292,False,True,"Bar, European, British, Healthy, Dining bars",NaN,"Reservations, Outdoor Seating, Seating, Serves...",15.0,4.0,7.0,11.0,279.0
3,2213156,The Black Dog Freehouse,733,4.5,$$ - $$$,False,51.488724,-0.120889,False,False,"Bar, British, Pub","Vegetarian friendly, Vegan options, Gluten fre...","Reservations, Outdoor Seating, Seating, Highch...",19.0,23.0,47.0,216.0,428.0
4,27930238,Fitz's Brasserie,48,5.0,$$ - $$$,False,51.522408,-0.124933,False,False,"Steakhouse, British, Contemporary",Vegetarian friendly,"Reservations, Seating, Wheelchair Accessible, ...",0.0,1.0,1.0,2.0,44.0


Nan в diets это не пропущенное значение, а признак "меню не является диетическим" по смыслу. Добавим

In [192]:
data['diets']=data['diets'].fillna('The menu does not match the diets')

Аналогично для is_sponsored :
это рестораны, которые рекомендует tripadvisor. Строки в которых этот пункт Nan, соответственно 0.

In [193]:
data['is_sponsored']=data['is_sponsored'].fillna('False')

Остальные Nan - отсутствие информации. Удалим строки с ними.

In [194]:
data=data.dropna()
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9077 entries, 0 to 10849
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                9077 non-null   object 
 1   name              9077 non-null   object 
 2   reviews           9077 non-null   int64  
 3   rating            9077 non-null   float64
 4   price_range       9077 non-null   object 
 5   is_sponsored      9077 non-null   object 
 6   latitude          9077 non-null   float64
 7   longitude         9077 non-null   float64
 8   has_delivery      9077 non-null   object 
 9   is_premium        9077 non-null   object 
 10  cuisines          9077 non-null   object 
 11  diets             9077 non-null   object 
 12  dining_options    9077 non-null   object 
 13  reviews_rating_1  9077 non-null   float64
 14  reviews_rating_2  9077 non-null   float64
 15  reviews_rating_3  9077 non-null   float64
 16  reviews_rating_4  9077 non-null   float64
 17 

Вернемся к дубликатам. Теперь остались почти одинаковые строки т к NaN убрали. Удалим случайным образом дубликаты по id

In [196]:
data = data.drop_duplicates(subset='id')

In [197]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8946 entries, 0 to 10737
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                8946 non-null   object 
 1   name              8946 non-null   object 
 2   reviews           8946 non-null   int64  
 3   rating            8946 non-null   float64
 4   price_range       8946 non-null   object 
 5   is_sponsored      8946 non-null   object 
 6   latitude          8946 non-null   float64
 7   longitude         8946 non-null   float64
 8   has_delivery      8946 non-null   object 
 9   is_premium        8946 non-null   object 
 10  cuisines          8946 non-null   object 
 11  diets             8946 non-null   object 
 12  dining_options    8946 non-null   object 
 13  reviews_rating_1  8946 non-null   float64
 14  reviews_rating_2  8946 non-null   float64
 15  reviews_rating_3  8946 non-null   float64
 16  reviews_rating_4  8946 non-null   float64
 17 

Получили датасет без дубликатов и пропусков размером около 83% от исходного

Теперь добавим бинарный признак, упоминается ли ресторан в MICHELIN Guide

In [210]:
data_michelin=pd.read_csv('restaurants_Michelin.csv')

In [211]:
data_michelin.loc[333]='Chishuru' #один из ресторанов записался вместо названия колонки

In [212]:
data_michelin = data_michelin.rename(columns={'Chishuru': 'name'})

In [213]:
data_michelin['IsIn_MICHELIN_Guide']=1

In [214]:
data_michelin

,name,IsIn_MICHELIN_Guide
0,El Pastor,1
1,Bancone,1
2,Paradise,1
3,Cabotte,1
4,Alain Ducasse at The Dorchester,1
...,...,...
329,Pophams,1
330,Takahashi,1
331,Plates London,1
332,La Trompette,1


Объединим с исходной таблицей

In [221]:
data_result=pd.merge(left=data, right=data_michelin, on=['name'], how='left')
data_result=data_result.fillna(0)
data_result

<ipython-input-221-279bbf9d8599>:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_result=data_result.fillna(0)


,id,name,reviews,rating,price_range,is_sponsored,latitude,longitude,has_delivery,is_premium,cuisines,diets,dining_options,reviews_rating_1,reviews_rating_2,reviews_rating_3,reviews_rating_4,reviews_rating_5,IsIn_MICHELIN_Guide
0,2418348,Leicester Arms,602,4.5,$$ - $$$,True,51.510597,-0.137677,False,False,"British, Pub","Vegetarian friendly, Gluten free options","Reservations, Seating, Street Parking, Televis...",30.0,23.0,51.0,156.0,342.0,0.0
1,26452946,Swiss Butter,182,4.0,$$ - $$$,True,51.520620,-0.122808,False,True,"French, Steakhouse",The menu does not match the diets,"Seating, Wheelchair Accessible, Serves Alcohol...",21.0,15.0,14.0,22.0,110.0,0.0
2,25151198,Florattica Rooftop Bar London,316,4.5,$$ - $$$,False,51.512955,-0.075292,False,True,"Bar, European, British, Healthy, Dining bars",The menu does not match the diets,"Reservations, Outdoor Seating, Seating, Serves...",15.0,4.0,7.0,11.0,279.0,0.0
3,2213156,The Black Dog Freehouse,733,4.5,$$ - $$$,False,51.488724,-0.120889,False,False,"Bar, British, Pub","Vegetarian friendly, Vegan options, Gluten fre...","Reservations, Outdoor Seating, Seating, Highch...",19.0,23.0,47.0,216.0,428.0,0.0
4,27930238,Fitz's Brasserie,48,5.0,$$ - $$$,False,51.522408,-0.124933,False,False,"Steakhouse, British, Contemporary",Vegetarian friendly,"Reservations, Seating, Wheelchair Accessible, ...",0.0,1.0,1.0,2.0,44.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8941,27608173,Sushi Kyu,44,4.6,£££,False,51.512120,-0.134599,True,True,Japanese,The menu does not match the diets,"Beer, Counter Seating, Non-Smoking, Takeaway, ...",1.0,0.0,0.0,0.0,9.0,0.0
8942,32243704,Gallery,8192,4.7,££££,False,51.601208,-0.243339,False,False,"Contemporary British, Afternoon Tea, Contempor...","Vegan, Gluten-free Options","Bar/Lounge, Beer, Cocktails, Entertainment, Fu...",1.0,2.0,2.0,1.0,3.0,0.0
8943,28149215,Mister Nice,17,4.0,££££,False,51.511200,-0.147390,False,False,"Contemporary French, French",The menu does not match the diets,"Cocktails, Wine",0.0,0.0,0.0,1.0,0.0,0.0
8944,28017181,Capilungo,15,4.6,££,False,51.514233,-0.122250,False,False,Italian,The menu does not match the diets,"Beer, Cafe, Cocktails, Full Bar, Non-Smoking, ...",2.0,1.0,0.0,1.0,4.0,0.0


In [225]:
data_result = data_result.drop(columns=['id']) #столбец id больше не имеет смысла
data_result

,name,reviews,rating,price_range,is_sponsored,latitude,longitude,has_delivery,is_premium,cuisines,diets,dining_options,reviews_rating_1,reviews_rating_2,reviews_rating_3,reviews_rating_4,reviews_rating_5,IsIn_MICHELIN_Guide
0,Leicester Arms,602,4.5,$$ - $$$,True,51.510597,-0.137677,False,False,"British, Pub","Vegetarian friendly, Gluten free options","Reservations, Seating, Street Parking, Televis...",30.0,23.0,51.0,156.0,342.0,0.0
1,Swiss Butter,182,4.0,$$ - $$$,True,51.520620,-0.122808,False,True,"French, Steakhouse",The menu does not match the diets,"Seating, Wheelchair Accessible, Serves Alcohol...",21.0,15.0,14.0,22.0,110.0,0.0
2,Florattica Rooftop Bar London,316,4.5,$$ - $$$,False,51.512955,-0.075292,False,True,"Bar, European, British, Healthy, Dining bars",The menu does not match the diets,"Reservations, Outdoor Seating, Seating, Serves...",15.0,4.0,7.0,11.0,279.0,0.0
3,The Black Dog Freehouse,733,4.5,$$ - $$$,False,51.488724,-0.120889,False,False,"Bar, British, Pub","Vegetarian friendly, Vegan options, Gluten fre...","Reservations, Outdoor Seating, Seating, Highch...",19.0,23.0,47.0,216.0,428.0,0.0
4,Fitz's Brasserie,48,5.0,$$ - $$$,False,51.522408,-0.124933,False,False,"Steakhouse, British, Contemporary",Vegetarian friendly,"Reservations, Seating, Wheelchair Accessible, ...",0.0,1.0,1.0,2.0,44.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8941,Sushi Kyu,44,4.6,£££,False,51.512120,-0.134599,True,True,Japanese,The menu does not match the diets,"Beer, Counter Seating, Non-Smoking, Takeaway, ...",1.0,0.0,0.0,0.0,9.0,0.0
8942,Gallery,8192,4.7,££££,False,51.601208,-0.243339,False,False,"Contemporary British, Afternoon Tea, Contempor...","Vegan, Gluten-free Options","Bar/Lounge, Beer, Cocktails, Entertainment, Fu...",1.0,2.0,2.0,1.0,3.0,0.0
8943,Mister Nice,17,4.0,££££,False,51.511200,-0.147390,False,False,"Contemporary French, French",The menu does not match the diets,"Cocktails, Wine",0.0,0.0,0.0,1.0,0.0,0.0
8944,Capilungo,15,4.6,££,False,51.514233,-0.122250,False,False,Italian,The menu does not match the diets,"Beer, Cafe, Cocktails, Full Bar, Non-Smoking, ...",2.0,1.0,0.0,1.0,4.0,0.0


In [227]:
data_result.to_csv('merged.csv')